In [ ]:
#https://github.com/amitrajitbose/handwritten-digit-recognition/blob/master/handwritten_digit_recognition_CPU.ipynb
#https://towardsdatascience.com/handwritten-digit-mnist-pytorch-977b5338e627
!pip install pyDOE

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18184 sha256=466633d027f7fe37e578aec35893614ef04e4f82075fa9581d4aabd68da209ac
  Stored in directory: /root/.cache/pip/wheels/83/ce/8a/87b25c685bfeca1872d13b8dc101e087a9c6e3fb5ebb47022a
Successfully built pyDOE


In [ ]:
#imports
# Import necessary packages
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
import os
from google.colab import drive
from torchvision import datasets, transforms
from torch import nn
from torch import optim


import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from sklearn.model_selection import train_test_split


#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

# MNIST

In [ ]:
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset = datasets.MNIST('drive/My Drive/mnist/MNIST_data/', download=True, train=True, transform=transform)
valset = datasets.MNIST('drive/My Drive/mnist/MNIST_data/', download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True) # change batch size to 128 from 64
valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting drive/My Drive/mnist/MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to drive/My Drive/mnist/MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting drive/My Drive/mnist/MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to drive/My Drive/mnist/MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting drive/My Drive/mnist/MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to drive/My Drive/mnist/MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting drive/My Drive/mnist/MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to drive/My Drive/mnist/MNIST_data/MNIST/raw



In [ ]:
#output test
#print(len(list(trainloader)))
#batch count

938


In [ ]:
# define the MNIST model
# Layer details for the neural network
input_size = 784
hidden_sizes = [128, 64] # [10, 10]
output_size = 10

# Build a feed-forward network
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
  (5): LogSoftmax(dim=1)
)


In [ ]:
criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

logps = model(images)
loss = criterion(logps, labels)

In [ ]:

# Optimizers require the parameters to optimize and a learning rate
MNIST_optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# PINN SETUP

In [ ]:
#PINN model
#TODO define c
c = 1 #heuristic
class PINN(nn.Module):
  #https://github.com/omniscientoctopus/Physics-Informed-Neural-Networks/tree/main/PyTorch/Burgers'%20Equation
    ##Neural Network
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
        'activation function'
        self.activation = nn.Tanh()
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)]) 
        self.iter = 0 #For the Optimizer
        'Xavier Normal Initialization'
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
    'foward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        a = x.float()
        for i in range(len(layers)-2):  
            z = self.linears[i](a)              
            a = self.activation(z)    
        a = self.linears[-1](a)
        return a
    'Loss Functions'
    #Loss BC
    def lossBC(self,x_BC,y_BC):
      loss_BC=self.loss_function(self.forward(x_BC),y_BC)
    
      return loss_BC
    #Loss PDE
    def lossPDE(self,x_PDE):
      g=x_PDE.clone()
     # g.requires_grad=True #Enable differentiation #TODO REVIEW
      g.retain_grad()
      f=self.forward(g)
      f_x_t = autograd.grad(f,g,torch.ones([g.shape[0], 1]).to(device), retain_graph=True, create_graph=True)[0] #first derivative
      f_xx_tt = autograd.grad(f_x_t,g,torch.ones(g.shape).to(device), create_graph=True)[0]#second derivative
      f_t=f_x_t[:,[1]]# we select the 2nd element for t (the first one is x) (Remember the input X=[x,t]) 
      f_xx=f_xx_tt[:,[0]]# we select the 1st element for x (the second one is t) (Remember the input X=[x,t]) 
      f_x=f_x_t[:,[0]]
      f= -f_t + f_x + f_xx - c
      return self.loss_function(f,f_hat)

    def loss(self,x_BC,y_BC,x_PDE):
      loss_bc=self.lossBC(x_BC,y_BC)

      loss_pde=self.lossPDE(x_PDE)
      return loss_bc+loss_pde

    #Optimizer              X_train_Nu,Y_train_Nu,X_train_Nf                   
    def closure(self):
      PINN_optimizer.zero_grad()  
      loss = self.loss(X_train_Nu,Y_train_Nu,X_train_Nf)
      loss.backward()      
      self.iter += 1
      if self.iter % 100 == 0:
        loss2=self.lossBC(X_test,Y_test)
        print("Training Error:",loss.detach().cpu().numpy(),"---Testing Error:",loss2.detach().cpu().numpy())
      return loss   

In [ ]:
#learning params
steps=1000
lr=1e-3
layers = np.array([2,32,32,1]) # hidden layers
# To generate new data:
#x_min=-1
#x_max=1
#t_min=0
#t_max=1
#total_points_x=200
#total_points_t=100
#Nu: Number of training points # Nf: Number of collocation points (Evaluate PDE)
#Nu=100
#Nf=10000

In [ ]:
# PINN training on device
#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name()) 


cpu


In [ ]:
#compact PINN training method
pinn_loss_graph = []
def trainPINN(X_train,Y_train,collocation,act_loss):
  for i in range(steps):
    if i==0:
      print("Training Loss-----Test Loss")
    loss = PINN_model.loss(X_train,Y_train,collocation)# use mean squared error
    PINN_optimizer.zero_grad()
    loss.backward()
    PINN_optimizer.step()
    if i%(steps/10)==0:
      #with torch.no_grad():
       # test_loss=PINN.lossBC(X_test,Y_test)
      print("--------------------------PINN STEP--------------------------")
      print('#',i)
      print('PINN SELF LOSS COMPUTATION:',loss.detach().cpu().numpy(),'---')#,test_loss.detach().cpu().numpy())
      #print('PINN PREDICTION OF MNIST LOSS',PINN_model.forward(X_train))
      print('MNIST GROUND TRUTH LOSS:', act_loss)
      print('PINN AVG. PREDICTION OF MNIST LOSS:',torch.mean(PINN_model.forward(X_train)))
      pinn_loss_graph.append(loss.detach().cpu().numpy())
     

# Training

In [ ]:
#MNIST training
MNIST_optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
time0 = time()
epochs = 1
for e in range(epochs):
    running_loss = 0
    n=0
    weight_train = [] # all weights, used for collocation
    collocation = []
    iter_weights = [] # store 8192 weights per iteration
    idx_mat = [] # zeroes mat + n
    iter_loss = 0
    #idx=np.random.choice(X_train.shape[0],Nu,replace=False) # train
    for images, labels in trainloader:
        n+=1
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
    
        # Training pass
        MNIST_optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
        
        #backprop
        loss.backward()
        
        #update weights
        MNIST_optimizer.step()
       
        running_loss += loss.item()
        print('--------------------------MNIST TRAINING ITERATION--------------------------')
        print('Iteration {} - Training loss: {}'.format(n,running_loss/len(trainloader)))
        #print('weight mat', model[2].weight[0])
        
        # Collect data for PINN
        iter_weights.append(model[2].weight)
        idx_mat = torch.zeros((8192,1)) + n

        #print('weight mat',torch.flatten(model[2].weight).unsqueeze(1))
        #print('idx_mat',idx_mat)

        weight_train = torch.hstack(( (torch.flatten(model[2].weight).unsqueeze(1)), idx_mat))
        iter_loss = running_loss/len(trainloader)
        f_hat = torch.zeros(weight_train.shape[0],1)

        #print('weight mat',torch.flatten(model[2].weight).shape())
        #print('idx_mat',idx_mat.shape())

        weight_train=weight_train.float().to(device)#Training Points (BC)
        iter_loss=torch.Tensor([iter_loss]).float().to(device)#Training Points (BC)
        weight_train=weight_train.float().to(device)#Collocation Points
        f_hat = f_hat.to(device)#to minimize function

        # build PINN model
        PINN_model = PINN(layers)
        PINN_model.to(device)

        params = list(PINN_model.parameters())
        PINN_optimizer = torch.optim.Adam(PINN_model.parameters(),lr=lr,amsgrad=False)
        
        if n != 1:
          # loading from training checkpoint
          checkpoint = torch.load(PATH)
          PINN_model.load_state_dict(checkpoint['model_state_dict'])
          PINN_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        # Train PINN
        trainPINN(weight_train,iter_loss,weight_train,running_loss/len(trainloader))
        
        # saving model checkpoint
        PATH = 'results' + str(n) + '.pth'

        torch.save({
            'model_state_dict': PINN_model.state_dict(),
            'optimizer_state_dict': PINN_optimizer.state_dict(),
            }, PATH)
        print('--------------------------SAVED PINN CHECKPOINT--------------------------')
        print('Checkpoint #', str(n))
          

    else:
        print("Epoch {} - Training loss: {}".format(e+1, running_loss/len(trainloader)))
print("\nTraining Time (in minutes) =",(time()-time0)/60)

--------------------------MNIST TRAINING ITERATION--------------------------
Iteration 1 - Training loss: 0.0048665298836063475
Training Loss-----Test Loss
--------------------------PINN STEP--------------------------
# 0
PINN SELF LOSS COMPUTATION: 1.6700616 ---
MNIST GROUND TRUTH LOSS: 0.0048665298836063475
PINN AVG. PREDICTION OF MNIST LOSS: tensor(0.2062, grad_fn=<MeanBackward0>)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([8192, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Streaming output truncated to the last 5000 lines.
--------------------------PINN STEP--------------------------
# 900
PINN SELF LOSS COMPUTATION: 3.31841e-06 ---
MNIST GROUND TRUTH LOSS: 0.31087735822714213
PINN AVG. PREDICTION OF MNIST LOSS: tensor(0.3123, grad_fn=<MeanBackward0>)
--------------------------SAVED PINN CHECKPOINT--------------------------
Checkpoint # 67
--------------------------MNIST TRAINING ITERATION--------------------------
Iteration 68 - Training loss: 0.31514711522344335
Training Loss-----Test Loss
--------------------------PINN STEP--------------------------
# 0
PINN SELF LOSS COMPUTATION: 0.00013730343 ---
MNIST GROUND TRUTH LOSS: 0.31514711522344335
PINN AVG. PREDICTION OF MNIST LOSS: tensor(0.3419, grad_fn=<MeanBackward0>)
--------------------------PINN STEP--------------------------
# 100
PINN SELF LOSS COMPUTATION: 3.5815865e-06 ---
MNIST GROUND TRUTH LOSS: 0.31514711522344335
PINN AVG. PREDICTION OF MNIST LOSS: tensor(0.3155, grad_fn=<MeanBackward0>)
---

In [ ]:
t = torch.flatten(model[2].weight)
print(t)
print(len(t))
#print(model[2].weight)
#print(len(model[2].weight))

tensor([-0.0065,  0.0633,  0.0408,  ...,  0.0657,  0.1179,  0.0986],
       grad_fn=<ReshapeAliasBackward0>)
8192


In [ ]:
#size testing
import numpy as np

V = np.zeros((8192,1)) + 4
K = np.zeros((8192,1))
Z = np.hstack((V,K))
print(Z)

[[4. 0.]
 [4. 0.]
 [4. 0.]
 ...
 [4. 0.]
 [4. 0.]
 [4. 0.]]
